# Installation

In [1]:
!pip install huggingface_hub==0.23.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.53.0 requires huggingface-hub<1.0,>=0.30.0, but you have huggingface-hub 0.23.2 which is incompatible.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.2 which is incompatible.
gradio 5.31.0 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.23.2 which is incompatible.
diffusers 0.34.0 requires huggingface-hub>=0.27.0, but you have huggingface-hub 0.23.2 which is incompatible.


In [2]:
# clone nemo
!git clone https://github.com/AI4Bharat/NeMo.git -b nemo-v2

# conda install -c nvidia cuda-nvprof=12.2 # Cuda version
!pip install packaging
!pip install huggingface_hub==0.23.2

# install NeMo
%cd NeMo
!bash reinstall.sh

Cloning into 'NeMo'...
remote: Enumerating objects: 79519, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 79519 (delta 186), reused 178 (delta 162), pack-reused 79295 (from 3)
Receiving objects: 100% (79519/79519), 141.13 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (57717/57717), done.
/content/NeMo
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Uninstalling stuff
Installing nemo
Obtaining file:///content/NeMo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Instal

# Necessary imports

In [ ]:
import torch
import soundfile as sf
import nemo.collections.asr as nemo_asr
import IPython

# Loading the model
Once you download the .nemo checkpoint, you can load it here

In [ ]:
!wget https://huggingface.co/ai4bharat/indicwhisper-medium/resolve/main/indicwhisper-medium.nemo -O checkpoint.nemo

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
#hf_

In [ ]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="ai4bharat/indicwhisper-medium", filename="indicwhisper-medium.nemo")

In [ ]:
# download and load model
!wget <NEMO_MODEL_PATH> -O checkpoint.nemo

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = nemo_asr.models.EncDecCTCModel.restore_from(restore_path='checkpoint.nemo')
model.eval() # inference mode
model = model.to(device) # transfer model to device

#Downloading a sample hindi audio
The following snippet downloads a test audio, downsamples it and makes it ready for inference

In [ ]:
# download and load audio
MEDIA_URL = "https://objectstore.e2enetworks.net/indic-asr-public/sample_audio.wav"
!wget $MEDIA_URL -O sample_audio.wav

# downsample, single channel
!ffmpeg -i sample_audio.wav -ac 1 -ar 16000 sample_audio_infer_ready.wav

In [ ]:
IPython.display.Audio('sample_audio_infer_ready.wav')

#Inference using CTC-Decoder

In [ ]:
model.cur_decoder = "ctc"
ctc_text = model.transcribe(['sample_audio_infer_ready.wav'], batch_size=1,logprobs=False, language_id='hi')[0]

Inference using RNN-T Decoder

In [ ]:
print(ctc_text)

In [ ]:
model.cur_decoder = "rnnt"
rnnt_text = model.transcribe(['sample_audio_infer_ready.wav'], batch_size=1, language_id='hi')[0]

In [ ]:
print(rnnt_text)